## Paquets i dades originals ## 

In [296]:
import json
import pandas as pd
from datetime import datetime
from listas import disfagia_list, PA_list, P_list, Main_respiratory_infections_list, LRTI_list, COPD_exacerbations_list, Pulmonary_fibrosis_fibrotorax_list
from auxiliary_functions import valores_codigos, contar_diccionarios, dias_ingreso_total, asignar_intervalo_edad, sumar_barthel, sumar_emina, obtenir_ultim_resultat, obtenir_valor_promedio, canadenca_comparada, disfagia_mecvvs, extraer_valor_clave, extraer_valor_clave_simple

taula = pd.read_pickle('./data/processed/dataframe.pkl')
import scipy.stats as stats
from scipy.stats import ttest_ind
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu
from scipy.stats import chi2_contingency


Prèviament, he tractat les dades creant noves columnes que classifiquen millor tota la informació disponible

In [297]:
taula.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,FGE CDK-EPI,pes més antic,pes més nou,data pes més antic,data primer mecvv,pes coincident primer mecvv,perdua pes entre ingressos,perdua pes total,fecha más antigua pneumonia,dias entre primer ICD pneumonia y primer MECVV positivo
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,None,None,None,None,NaT,None,None,None,2020-06-18,NaN
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,>90,70,53.7,2022-07-06,NaT,None,None,16.3,2022-09-29,NaN
2,96,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-09 00...",[],...,None,None,None,None,NaT,None,None,None,2020-07-08,NaN
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,2020-12-04,None,None,None,2020-12-02,2.0
4,93,M,0,"[{'date': '20220208', 'name': 'UREA Sèrum', 'l...","[{'imc': '0', 'anorexia': '1', 'perduaPes': '1...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '49.900', 'data': '2022-06-02'}, {'...","[{'codi': 'N02AA', 'dataInici': '2022-02-01 00...","[{'data': '20220330172303', 'disfagiaConeguda'...",...,Proves afegides a un altre petició per extracc...,49.900,49.900,2022-06-02,NaT,None,None,0.0,2022-08-07,NaN


Distribuiré les dades en 3 grups (més un grup que conté la totalitat de dades):
- **AMB_PA**: aquest grup està compost per pacients que contenen el codi ICD de pneumònia aspirativa al seu historial.
- **AMB_PA_MECVV**: aquest grup conté els pacients que compleixen les següents característiques: 
    - Han donat positiu al MECVV (és a dir, que tenen disfàgia i una alteració en la seguretat o en l'eficàcia)
    - Han sigut diagnosticats amb el codi ICD de pneumònia
    - Han passat menys de 30 dies entre el diagnòstic amb l'ICD de pneumònia i el resultat positiu al MECVV
- **SENSE_PA**: aquest grup està format per pacients que contenen el codi ICD de pneumònia o bé:
    - Contenen el codi ICD de pneumònia
    - Han donat positiu al MECVV 
    - Han passat més de 30 dies entre el diagnòstic amb l'ICD de pneumònia i  el resultat positiu del MECVV 
- **TOTAL**: aquest grup conté tots els pacients de l'estudi *#TODO: hacer descriptiva de este grupo*

# AMB_PA 

In [298]:
AMB_PA = taula[taula["PA_diagnosticada"] == 1.0]
AMB_PA.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,FGE CDK-EPI,pes més antic,pes més nou,data pes més antic,data primer mecvv,pes coincident primer mecvv,perdua pes entre ingressos,perdua pes total,fecha más antigua pneumonia,dias entre primer ICD pneumonia y primer MECVV positivo
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,None,None,None,None,NaT,None,None,None,2020-06-18,NaN
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,>90,70,53.7,2022-07-06,NaT,None,None,16.3,2022-09-29,NaN
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,2020-12-04,None,None,None,2020-12-02,2.0
6,101,F,0,[],[],"[{'activitat': '3', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'C09AA', 'dataInici': '2021-08-21 00...","[{'data': '20210819124116', 'disfagia': 'S', '...",...,None,None,None,None,2021-08-19,None,None,None,2021-08-17,2.0
12,86,M,0,[],[],"[{'activitat': '1', 'estatMental': '1', 'humit...",[],"[{'valor': '46.4', 'data': '2020-09-09'}]","[{'codi': 'G04CA', 'dataInici': '2020-09-04 00...","[{'data': '20200907110400', 'disfagia': 'S', '...",...,None,46.4,46.4,2020-09-09,2020-09-07,46.4,0.0,0.0,2020-09-04,3.0


# AMB_PA_MECVV

In [299]:
AMB_PA_MECVV = taula[
    (taula["dias entre primer ICD pneumonia y primer MECVV positivo"] < 30)] 
AMB_PA_MECVV.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,FGE CDK-EPI,pes més antic,pes més nou,data pes més antic,data primer mecvv,pes coincident primer mecvv,perdua pes entre ingressos,perdua pes total,fecha más antigua pneumonia,dias entre primer ICD pneumonia y primer MECVV positivo
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,2020-12-04,None,None,None,2020-12-02,2.0
6,101,F,0,[],[],"[{'activitat': '3', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'C09AA', 'dataInici': '2021-08-21 00...","[{'data': '20210819124116', 'disfagia': 'S', '...",...,None,None,None,None,2021-08-19,None,None,None,2021-08-17,2.0
12,86,M,0,[],[],"[{'activitat': '1', 'estatMental': '1', 'humit...",[],"[{'valor': '46.4', 'data': '2020-09-09'}]","[{'codi': 'G04CA', 'dataInici': '2020-09-04 00...","[{'data': '20200907110400', 'disfagia': 'S', '...",...,None,46.4,46.4,2020-09-09,2020-09-07,46.4,0.0,0.0,2020-09-04,3.0
27,100,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],"[{'valor': '0', 'data': '2021-09-14'}, {'valor...","[{'codi': 'C08CA', 'dataInici': '2020-11-23 00...","[{'data': '20201125131827', 'disfagia': 'S', '...",...,None,41.100,0,2021-01-20,2020-11-25,None,None,41.1,2020-12-02,7.0
28,95,F,0,"[{'date': '20220526', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],"[{'valor': '0', 'data': '2022-05-30'}]","[{'codi': 'C10AA', 'dataInici': '2020-11-18 00...","[{'data': '20220529115841', 'disfagia': 'S', '...",...,48,0,0,2022-05-30,2022-05-29,0,0.0,0.0,2022-05-23,6.0


# SENSE_PA

In [587]:
SENSE_PA = taula[
           (taula["dias entre primer ICD pneumonia y primer MECVV positivo"] > 30)  (taula['P_diagnosticada'] == 1.0)] 
SENSE_PA.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,FGE CDK-EPI,pes més antic,pes més nou,data pes més antic,data primer mecvv,pes coincident primer mecvv,perdua pes entre ingressos,perdua pes total,fecha más antigua pneumonia,dias entre primer ICD pneumonia y primer MECVV positivo
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,None,None,None,None,NaT,None,None,None,2020-06-18,NaN
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,>90,70,53.7,2022-07-06,NaT,None,None,16.3,2022-09-29,NaN
2,96,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-09 00...",[],...,None,None,None,None,NaT,None,None,None,2020-07-08,NaN
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,2020-12-04,None,None,None,2020-12-02,2.0
4,93,M,0,"[{'date': '20220208', 'name': 'UREA Sèrum', 'l...","[{'imc': '0', 'anorexia': '1', 'perduaPes': '1...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '49.900', 'data': '2022-06-02'}, {'...","[{'codi': 'N02AA', 'dataInici': '2022-02-01 00...","[{'data': '20220330172303', 'disfagiaConeguda'...",...,Proves afegides a un altre petició per extracc...,49.900,49.900,2022-06-02,NaT,None,None,0.0,2022-08-07,NaN


In [589]:
#TODO: si quito el | (taula['P_diagnosticada'] == 1.0) se me quedan en 8 pacientes 

# TOTAL

In [301]:
TOTAL = taula
taula.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,FGE CDK-EPI,pes més antic,pes més nou,data pes més antic,data primer mecvv,pes coincident primer mecvv,perdua pes entre ingressos,perdua pes total,fecha más antigua pneumonia,dias entre primer ICD pneumonia y primer MECVV positivo
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,None,None,None,None,NaT,None,None,None,2020-06-18,NaN
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,>90,70,53.7,2022-07-06,NaT,None,None,16.3,2022-09-29,NaN
2,96,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-09 00...",[],...,None,None,None,None,NaT,None,None,None,2020-07-08,NaN
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,2020-12-04,None,None,None,2020-12-02,2.0
4,93,M,0,"[{'date': '20220208', 'name': 'UREA Sèrum', 'l...","[{'imc': '0', 'anorexia': '1', 'perduaPes': '1...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '49.900', 'data': '2022-06-02'}, {'...","[{'codi': 'N02AA', 'dataInici': '2022-02-01 00...","[{'data': '20220330172303', 'disfagiaConeguda'...",...,Proves afegides a un altre petició per extracc...,49.900,49.900,2022-06-02,NaT,None,None,0.0,2022-08-07,NaN


Ara que tinc totes les dades classificades en aquests grups, faré una descriptiva. També compararé els diferents grups amb el pvalor mitjançant T-test, test de Mann-Whitney o test Xi-quadrat per mirar la seva distribució, la qual esperaré que sigui similar entre el grup AMB_PA i AMB_PA_MECVV, i diferent entre SENSE_PA i els altres 2 grups. Els criteris que seguiré per utilitzar un test o un altre és, primer, fer un test de Shapiro-Wilk per comprovar si segueixen una distribució normal, si és així, usaré el T-test, si no, utilitzaré el de Mann-Whitney. Finalment, si la columna conté valors categòrics com és el cas del sexe (F/M), faré servir el test Xi-quadrat.


## Nombre de pacients

In [588]:
num_AMB_PA = len(AMB_PA)
num_AMB_PA_MECVV= len(AMB_PA_MECVV)
num_SENSE_PA = len(SENSE_PA)
num_TOTAL = len(TOTAL)

print(num_AMB_PA, num_AMB_PA_MECVV, num_SENSE_PA, num_TOTAL)

17 8 49 50


#### Pvalor

##### Normalitat

In [303]:
stat, p_value = shapiro(range(num_AMB_PA))
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9621
Valor p = 0.6704
El nombre de files segueix una distribució normal.


In [304]:
stat, p_value = shapiro(range(num_AMB_PA_MECVV))
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9749
Valor p = 0.9332
El nombre de files segueix una distribució normal.


In [305]:
stat, p_value = shapiro(range(num_SENSE_PA))
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9556
Valor p = 0.0626
El nombre de files segueix una distribució normal.


In [306]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA.index, AMB_PA_MECVV.index, equal_var=False)
print(f"P-valor entre AMB_PA i AMB_PA_MECVV: {p_val_12:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

P-valor entre AMB_PA i AMB_PA_MECVV: 0.7033
No existeixen diferències significatives entre els 2 grups.


In [307]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA.index, SENSE_PA.index, equal_var=False)
print(f"P-valor entre AMB_PA i SENSE_PA: {p_val_13:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

P-valor entre AMB_PA i SENSE_PA: 0.7664
No existeixen diferències significatives entre els 2 grups.


In [308]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV.index, SENSE_PA.index, equal_var=False)
print(f"P-valor entre AMB_PA_MECVV i SENSE_PA: {p_val_12:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.7033
No existeixen diferències significatives entre els 2 grups.


## Edat

In [309]:
edad_AMB_PA = AMB_PA['edat'].mean()
desviacion_estandar_edad = AMB_PA['edat'].std()

resultado = f"{edad_AMB_PA:.2f} ± {desviacion_estandar_edad:.2f}"
print("Mitjana ± Desviació Estàndard de l'edat:", resultado)


Mitjana ± Desviació Estàndard de l'edat: 87.06 ± 8.99


In [310]:
edad_AMB_PA_MECVV = AMB_PA_MECVV['edat'].mean()
desviacion_estandar_edad = AMB_PA_MECVV['edat'].std()

resultado = f"{edad_AMB_PA_MECVV:.2f} ± {desviacion_estandar_edad:.2f}"
print("Mitjana ± Desviació Estàndard de l'edat:", resultado)

Mitjana ± Desviació Estàndard de l'edat: 91.50 ± 6.82


In [311]:
edad_SENSE_PA = SENSE_PA['edat'].mean()
desviacion_estandar_edad = SENSE_PA['edat'].std()

resultado = f"{edad_SENSE_PA:.2f} ± {desviacion_estandar_edad:.2f}"
print("Mitjana ± Desviació Estàndard de l'edat:", resultado)

Mitjana ± Desviació Estàndard de l'edat: 80.12 ± 17.01


#### Pvalor

##### Normalitat 

In [312]:
stat, p_value = shapiro(AMB_PA['edat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9410
Valor p = 0.3297
El nombre de files segueix una distribució normal.


In [313]:
stat, p_value = shapiro(AMB_PA_MECVV['edat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8304
Valor p = 0.0600
El nombre de files segueix una distribució normal.


In [314]:
stat, p_value = shapiro(SENSE_PA['edat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8539
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [315]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['edat'], AMB_PA_MECVV['edat'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 0.18896133816771837


In [316]:
stat, p_value_13 = mannwhitneyu(AMB_PA['edat'], SENSE_PA['edat'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.1815160884632735


In [317]:
stat_23, p_value_23 = mannwhitneyu(AMB_PA_MECVV['edat'], SENSE_PA['edat'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.051903012711427096


## Sexe

In [318]:
sexe_AMB_PA = AMB_PA['sexe'].value_counts()
sexe_AMB_PA_MECVV = AMB_PA_MECVV['sexe'].value_counts()
sexe_SENSE_PA = SENSE_PA['sexe'].value_counts()

In [319]:
if 'F' in sexe_AMB_PA:
    percent_female1 = (sexe_AMB_PA['F'] / len(AMB_PA)) * 100
    print(f"Percentatge de sexe femení en AMB_PA: {percent_female1:.2f}%")
else:
    print("No hi ha dades de sexe femení en AMB_PA")

Percentatge de sexe femení en AMB_PA: 47.06%


In [320]:
if 'F' in sexe_AMB_PA_MECVV:
    percent_female2 = (sexe_AMB_PA_MECVV['F'] / len(AMB_PA)) * 100
    print(f"Percentatge de sexe femení en AMB_PA_MECVV: {percent_female1:.2f}%")
else:
    print("No hi ha dades de sexe femení en AMB_PA_MECVV")

Percentatge de sexe femení en AMB_PA_MECVV: 47.06%


In [321]:
if 'F' in sexe_SENSE_PA:
    percent_female3 = (sexe_SENSE_PA['F'] / len(SENSE_PA)) * 100
    print(f"Percentatge de sexe femení en SENSE_PA: {percent_female1:.2f}%")
else:
    print("No hi ha dades de sexe femení en SENSE_PA")


Percentatge de sexe femení en SENSE_PA: 47.06%


#### Pvalor

##### Normalitat

Al ser una variable categòrica no es pot comprovar la seva normalitat. Realitzaré directament el test Xi-quadrat

In [322]:
contingency_table_AMB_PA_AMB_PA_MECVV = pd.crosstab(index=AMB_PA['sexe'], columns=AMB_PA_MECVV['sexe'])

chi2_AMB_PA_AMB_PA_MECVV, p_value_AMB_PA_AMB_PA_MECVV, dof_AMB_PA_AMB_PA_MECVV, expected_AMB_PA_AMB_PA_MECVV = chi2_contingency(contingency_table_AMB_PA_AMB_PA_MECVV)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_AMB_PA_AMB_PA_MECVV)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_AMB_PA_AMB_PA_MECVV)

if p_value_AMB_PA_AMB_PA_MECVV < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 2.34375
P-valor entre AMB_PA i AMB_PA_MECVV: 0.12578642463893994
No hi ha diferències significatives entre les distribucions de sexe en AMB_PA i AMB_PA_MECVV.


In [323]:
contingency_table_AMB_PA_SENSE_PA = pd.crosstab(index=AMB_PA['sexe'], columns=SENSE_PA['sexe'])

chi2_AMB_PA_SENSE_PA, p_value_AMB_PA_SENSE_PA, dof_AMB_PA_SENSE_PA, expected_AMB_PA_SENSE_PA = chi2_contingency(contingency_table_AMB_PA_SENSE_PA)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_AMB_PA_SENSE_PA)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_AMB_PA_SENSE_PA)

if p_value_AMB_PA_SENSE_PA < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en AMB_PA i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 13.223042052469136
P-valor entre AMB_PA i SENSE_PA: 0.00027652843917458906
Hi ha diferències significatives entre les distribucions de sexe en AMB_PA i SENSE_PA.


In [324]:
contingency_table_AMB_PA_MECVV_SENSE_PA = pd.crosstab(index=AMB_PA_MECVV['sexe'], columns=SENSE_PA['sexe'])

chi2_AMB_PA_MECVV_SENSE_PA, p_value_AMB_PA_MECVV_SENSE_PA, dof_AMB_PA_MECVV_SENSE_PA, expected_AMB_PA_MECVV_SENSE_PA = chi2_contingency(contingency_table_AMB_PA_MECVV_SENSE_PA)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_AMB_PA_MECVV_SENSE_PA)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_AMB_PA_MECVV_SENSE_PA)

if p_value_AMB_PA_MECVV_SENSE_PA < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en AMB_PA_MECVV i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA: 3.5555555555555554
P-valor entre AMB_PA_MECVV i SENSE_PA: 0.05934643879191998
No hi ha diferències significatives entre les distribucions de sexe en AMB_PA_MECVV i SENSE_PA.


## Ingressos (admissions)

In [325]:
promedio_ingresos = AMB_PA['Num_ingresos'].mean()
desviacion_estandar_ingresos = AMB_PA['Num_ingresos'].std()

resultado = f"{promedio_ingresos:.2f} ± {desviacion_estandar_ingresos:.2f}"
print("Mitjana ± Desviació Estàndard del nombre d'ingressos:", resultado)

Mitjana ± Desviació Estàndard del nombre d'ingressos: 5.59 ± 3.37


In [326]:
promedio_ingresos = AMB_PA_MECVV['Num_ingresos'].mean()
desviacion_estandar_ingresos = AMB_PA_MECVV['Num_ingresos'].std()

resultado = f"{promedio_ingresos:.2f} ± {desviacion_estandar_ingresos:.2f}"
print("Mitjana ± Desviació Estàndard del nombre d'ingressos:", resultado)

Mitjana ± Desviació Estàndard del nombre d'ingressos: 5.88 ± 1.64


In [327]:
promedio_ingresos = SENSE_PA['Num_ingresos'].mean()
desviacion_estandar_ingresos = SENSE_PA['Num_ingresos'].std()

resultado = f"{promedio_ingresos:.2f} ± {desviacion_estandar_ingresos:.2f}"
print("Mitjana ± Desviació Estàndard del nombre d'ingressos:", resultado)

Mitjana ± Desviació Estàndard del nombre d'ingressos: 6.69 ± 5.13


#### Pvalor

##### Normalitat

In [328]:
stat, p_value = shapiro(AMB_PA['Num_ingresos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8008
Valor p = 0.0021
El nombre de files no segueix una distribució normal.


In [329]:
stat, p_value = shapiro(AMB_PA_MECVV['Num_ingresos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9122
Valor p = 0.3695
El nombre de files segueix una distribució normal.


In [330]:
stat, p_value = shapiro(SENSE_PA['Num_ingresos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.7277
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [331]:
stat, p_value_12 = mannwhitneyu(AMB_PA['Num_ingresos'], AMB_PA_MECVV['Num_ingresos'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.27403132109149697


In [332]:
stat, p_value_13 = mannwhitneyu(AMB_PA['Num_ingresos'], SENSE_PA['Num_ingresos'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.514931142191354


In [333]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['Num_ingresos'], SENSE_PA['Num_ingresos'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.6934523264142625


## Disfàgia

In [334]:
disf_AMB_PA= (AMB_PA['DO_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de disfàgia: {disf_AMB_PA:.2f}%")

Percentatge de disfàgia: 47.06%


In [335]:
disf_AMB_PA_MECVV = (AMB_PA_MECVV['DO_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de disfàgia: {disf_AMB_PA_MECVV:.2f}%")

Percentatge de disfàgia: 50.00%


In [336]:
disf_SENSE_PA= (SENSE_PA['DO_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de disfàgia: {disf_SENSE_PA:.2f}%")

Percentatge de disfàgia: 18.37%


#### Pvalor

##### Normalitat

In [337]:
stat, p_value = shapiro(AMB_PA['DO_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.6418
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [338]:
stat, p_value = shapiro(AMB_PA_MECVV['DO_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.6647
Valor p = 0.0009
El nombre de files no segueix una distribució normal.


In [339]:
stat, p_value = shapiro(SENSE_PA['DO_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.4716
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [340]:
stat, p_value_12 = mannwhitneyu(AMB_PA['DO_diagnosticada'], AMB_PA_MECVV['DO_diagnosticada'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.9196291134973918


In [341]:
stat, p_value_13 = mannwhitneyu(AMB_PA['DO_diagnosticada'], SENSE_PA['DO_diagnosticada'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.02125060274444335


In [342]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['DO_diagnosticada'], SENSE_PA['DO_diagnosticada'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.05191606463404439


## Index de Barthel

### Promedio

In [343]:
promedio_barthel = AMB_PA['Barthel_resultados'].mean()
desviacion_estandar_barthel = AMB_PA['Barthel_resultados'].std()

resultado = f"{promedio_barthel:.2f} ± {desviacion_estandar_barthel:.2f}"
print("Mitjana ± Desviació Estàndard del Barthel:", resultado)

Mitjana ± Desviació Estàndard del Barthel: 22.65 ± 24.63


In [344]:
promedio_barthel = AMB_PA_MECVV['Barthel_resultados'].mean()
desviacion_estandar_barthel = AMB_PA_MECVV['Barthel_resultados'].std()

resultado = f"{promedio_barthel:.2f} ± {desviacion_estandar_barthel:.2f}"
print("Mitjana ± Desviació Estàndard del Barthel:", resultado)

Mitjana ± Desviació Estàndard del Barthel: 13.75 ± 18.27


In [345]:
promedio_barthel = SENSE_PA['Barthel_resultados'].mean()
desviacion_estandar_barthel = SENSE_PA['Barthel_resultados'].std()

resultado = f"{promedio_barthel:.2f} ± {desviacion_estandar_barthel:.2f}"
print("Mitjana ± Desviació Estàndard del Barthel:", resultado)

Mitjana ± Desviació Estàndard del Barthel: 48.88 ± 37.42


#### Pvalor

##### Normalitat

In [346]:
stat, p_value = shapiro(AMB_PA['Barthel_resultados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8517
Valor p = 0.0114
El nombre de files no segueix una distribució normal.


In [347]:
stat, p_value = shapiro(AMB_PA_MECVV['Barthel_resultados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8000
Valor p = 0.0286
El nombre de files no segueix una distribució normal.


In [348]:
stat, p_value = shapiro(SENSE_PA['Barthel_resultados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8858
Valor p = 0.0002
El nombre de files no segueix una distribució normal.


In [349]:
stat, p_value_12 = mannwhitneyu(AMB_PA['Barthel_resultados'], AMB_PA_MECVV['Barthel_resultados'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.47528505928431597


In [350]:
stat, p_value_13 = mannwhitneyu(AMB_PA['Barthel_resultados'], SENSE_PA['Barthel_resultados'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.012373091641540783


In [351]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['Barthel_resultados'], SENSE_PA['Barthel_resultados'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.014652857928272206


### Valors de dependència

##### AMB_PA

In [352]:
indepe_AMB_PA = (AMB_PA['Barthel_resultados'] == 100.0).mean() * 100
print(f"Percentatge de indepe_AMB_PA: {indepe_AMB_PA:.2f}%")

Percentatge de indepe_AMB_PA: 0.00%


In [353]:
mod_AMB_PA = (AMB_PA['Barthel_resultados'].between(61, 95)).mean() * 100
print(f"Percentatge de mod_AMB_PA: {mod_AMB_PA:.2f}%")

Percentatge de mod_AMB_PA: 5.88%


In [354]:
sev_AMB_PA = (AMB_PA['Barthel_resultados'].between(21, 60)).mean() * 100
print(f"Percentatge de sev_AMB_PA: {sev_AMB_PA:.2f}%")

Percentatge de sev_AMB_PA: 41.18%


In [355]:
total_AMB_PA = (AMB_PA['Barthel_resultados'] <= 20.0 ).mean() * 100
print(f"Percentatge de total_AMB_PA: {total_AMB_PA:.2f}%")

Percentatge de total_AMB_PA: 52.94%


##### AMB_PA_MECVV

In [356]:
indepe_AMB_PA_MECVV = (AMB_PA_MECVV['Barthel_resultados'] == 100.0).mean() * 100
print(f"Percentatge de indepe_AMB_PA_MECVV: {indepe_AMB_PA_MECVV:.2f}%")

Percentatge de indepe_AMB_PA_MECVV: 0.00%


In [357]:
mod_AMB_PA_MECVV = (AMB_PA_MECVV['Barthel_resultados'].between(61,95)).mean() * 100
print(f"Percentatge de mod_AMB_PA_MECVV: {mod_AMB_PA_MECVV:.2f}%")

Percentatge de mod_AMB_PA_MECVV: 0.00%


In [358]:
sev_AMB_PA_MECVV = (AMB_PA_MECVV['Barthel_resultados'].between(21,60)).mean() * 100
print(f"Percentatge de sev_AMB_PA_MECVV: {sev_AMB_PA_MECVV:.2f}%")

Percentatge de sev_AMB_PA_MECVV: 25.00%


In [359]:
total_AMB_PA_MECVV = (AMB_PA_MECVV['Barthel_resultados'] <= 20.0).mean() * 100
print(f"Percentatge de total_AMB_PA_MECVV: {total_AMB_PA_MECVV:.2f}%")

Percentatge de total_AMB_PA_MECVV: 75.00%


##### SENSE_PA

In [360]:
indepe_SENSE_PA = (SENSE_PA['Barthel_resultados'] == 100.0).mean() * 100
print(f"Percentatge de indepe_SENSE_PA: {indepe_SENSE_PA:.2f}%")

Percentatge de indepe_SENSE_PA: 20.41%


In [361]:
mod_SENSE_PA = (SENSE_PA['Barthel_resultados'].between(61,95)).mean() * 100
print(f"Percentatge de mod_SENSE_PA: {mod_SENSE_PA:.2f}%")

Percentatge de mod_SENSE_PA: 18.37%


In [362]:
sev_SENSE_PA = (SENSE_PA['Barthel_resultados'].between(21,60)).mean() * 100
print(f"Percentatge de sev_SENSE_PA: {sev_SENSE_PA:.2f}%")

Percentatge de sev_SENSE_PA: 28.57%


In [363]:
total_SENSE_PA = (SENSE_PA['Barthel_resultados'] <= 20).mean() * 100
print(f"Percentatge de total_SENSE_PA: {total_SENSE_PA:.2f}%")

Percentatge de total_SENSE_PA: 32.65%


#### Pvalor

##### Independència

In [364]:
stat_12, p_value_12 = mannwhitneyu(AMB_PA['Barthel_resultados'] == 100.0, AMB_PA_MECVV['Barthel_resultados'] == 100.0, alternative='two-sided')
print("P-valor entre indepe_AMB_PA i indepe_AMB_PA_MECVV:", p_value_12)

P-valor entre indepe_AMB_PA i indepe_AMB_PA_MECVV: 1.0


In [365]:
stat_13, p_value_13 = mannwhitneyu(AMB_PA['Barthel_resultados'] == 100.0, SENSE_PA['Barthel_resultados'] == 100.0, alternative='two-sided')
print("P-valor entre indepe_AMB_PA i indepe_SENSE_PA:", p_value_13)

P-valor entre indepe_AMB_PA i indepe_SENSE_PA: 0.04605091215969595


In [366]:
stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Barthel_resultados'] == 100.0, SENSE_PA['Barthel_resultados'] == 100.0, equal_var=False)
print("P-valor entre indepe_AMB_PA_MECVV i SENSE_PA:", p_value_23)

P-valor entre indepe_AMB_PA_MECVV i SENSE_PA: 0.014652857928272206


##### Moderada

In [367]:
stat_12, p_value_12 = mannwhitneyu(AMB_PA['Barthel_resultados'].between(61, 95), AMB_PA_MECVV['Barthel_resultados'].between(61, 95), alternative='two-sided')
print("P-valor entre mod_AMB_PA i mod_AMB_PA_MECVV:", p_value_12)

P-valor entre mod_AMB_PA i mod_AMB_PA_MECVV: 0.5483429354628137


In [368]:
stat_13, p_value_13 = mannwhitneyu(AMB_PA['Barthel_resultados'].between(61, 95), SENSE_PA['Barthel_resultados'].between(61, 95), alternative='two-sided')
print("P-valor entre indepe_AMB_PA i mod_SENSE_PA:", p_value_13)

P-valor entre indepe_AMB_PA i mod_SENSE_PA: 0.22404485220846948


In [369]:
stat_23, p_value_23 = mannwhitneyu(AMB_PA_MECVV['Barthel_resultados'].between(61, 95), SENSE_PA['Barthel_resultados'].between(61, 95), alternative='two-sided')  
print("P-valor entre mod_AMB_PA_MECVV i mod_SENSE_PA :", p_value_23)

P-valor entre mod_AMB_PA_MECVV i mod_SENSE_PA : 0.19666119720150388


##### Severa

In [370]:
stat_12, p_value_12 = mannwhitneyu(AMB_PA['Barthel_resultados'].between(21,60), AMB_PA_MECVV['Barthel_resultados'].between(21,60), alternative='two-sided')
print("P-valor entre sev_AMB_PA i sev_AMB_PA_MECVV:", p_value_12)

P-valor entre sev_AMB_PA i sev_AMB_PA_MECVV: 0.4622499461255427


In [371]:
stat_13, p_value_13 = mannwhitneyu(AMB_PA['Barthel_resultados'].between(21,60), SENSE_PA['Barthel_resultados'].between(21,60), alternative='two-sided')
print("P-valor entre sev_AMB_PA i sev_SENSE_PA:", p_value_13)

P-valor entre sev_AMB_PA i sev_SENSE_PA: 0.3446368758082651


In [372]:
t_stat_23, p_value_23 = mannwhitneyu(AMB_PA_MECVV['Barthel_resultados'].between(21,60), SENSE_PA['Barthel_resultados'].between(21,60), alternative='two-sided' )
print("P-valor entre sev_AMB_PA_MECVV i sev_SENSE_PA:", p_value_23)

P-valor entre sev_AMB_PA_MECVV i sev_SENSE_PA: 0.8478664138880776


##### Total

In [373]:
stat_12, p_value_12 = mannwhitneyu(AMB_PA['Barthel_resultados'] <= 20.0, AMB_PA_MECVV['Barthel_resultados'] <= 20.0, alternative='two-sided')
print("P-valor entre total_AMB_PA i total_AMB_PA_MECVV:", p_value_12)

P-valor entre total_AMB_PA i total_AMB_PA_MECVV: 0.31988618684011016


In [374]:
stat_13, p_value_13 = mannwhitneyu(AMB_PA['Barthel_resultados'] <= 20.0, SENSE_PA['Barthel_resultados'] <= 20.0, alternative='two-sided')
print("P-valor entre total_AMB_PA i total_SENSE_PA:", p_value_13)

P-valor entre total_AMB_PA i total_SENSE_PA: 0.14269651822055499


In [375]:
stat_23, p_value_23 = mannwhitneyu(AMB_PA_MECVV['Barthel_resultados'] <= 20.0, SENSE_PA['Barthel_resultados'] <= 20.0, alternative='two-sided')
print("P-valor entre total_AMB_PA_MECVV i total_SENSE_PA:", p_value_23)

P-valor entre total_AMB_PA_MECVV i total_SENSE_PA: 0.0246114402001047


## Index del MNA

Primer convertiré la columna a tipus numèric ja que els valors que conté són de tipus objecte i amb aquest tipus no puc calcular la mitjana ni la desviació estàndard.

In [376]:
AMB_PA.loc[:, 'mna_resultats'] = pd.to_numeric(AMB_PA['mna_resultats'], errors='coerce')
AMB_PA_MECVV.loc[:, 'mna_resultats'] = pd.to_numeric(AMB_PA_MECVV['mna_resultats'], errors='coerce')
SENSE_PA.loc[:, 'mna_resultats'] = pd.to_numeric(SENSE_PA['mna_resultats'], errors='coerce')

### Mitjana

In [377]:
promedio_mna = AMB_PA['mna_resultats'].mean()
desviacion_estandar_mna = AMB_PA['mna_resultats'].std()

resultado = f"{promedio_mna:.2f} ± {desviacion_estandar_mna:.2f}"
print("Mitjana ± Desviació Estándard del mna:", resultado)

Mitjana ± Desviació Estándard del mna: 0.00 ± 0.00


In [378]:
promedio_mna = AMB_PA_MECVV['mna_resultats'].mean()
desviacion_estandar_mna = AMB_PA_MECVV['mna_resultats'].std()

resultado = f"{promedio_mna:.2f} ± {desviacion_estandar_mna:.2f}"
print("Mitjana ± Desviació Estándard del mna:", resultado)

Mitjana ± Desviació Estándard del mna: 0.00 ± 0.00


In [379]:
promedio_mna = SENSE_PA['mna_resultats'].mean()
desviacion_estandar_mna = SENSE_PA['mna_resultats'].std()

resultado = f"{promedio_mna:.2f} ± {desviacion_estandar_mna:.2f}"
print("Mitjana ± Desviació Estàndard del mna:", resultado)

Mitjana ± Desviació Estàndard del mna: 3.05 ± 5.92


#### Pvalor

##### Normalitat

In [380]:
stat, p_value = shapiro(AMB_PA['mna_resultats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [381]:
stat, p_value = shapiro(AMB_PA_MECVV['mna_resultats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [382]:
stat, p_value = shapiro(SENSE_PA['mna_resultats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [383]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['mna_resultats'], AMB_PA_MECVV['mna_resultats'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [384]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['mna_resultats'], SENSE_PA['mna_resultats'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [385]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['mna_resultats'], SENSE_PA['mna_resultats'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Valors de malnutrició

##### AMB_PA

In [386]:
norm_AMB_PA = AMB_PA['mna_resultats'].between(24, 30)
Percentatge_norm_AMB_PA = norm_AMB_PA.mean() * 100
num_observaciones_norm_AMB_PA = norm_AMB_PA.sum()

print(f"Percentatge de norm_AMB_PA: {Percentatge_norm_AMB_PA:.2f}%")
print(f"Nombre d'observacions en norm_AMB_PA: {num_observaciones_norm_AMB_PA}")

Percentatge de norm_AMB_PA: 0.00%
Nombre d'observacions en norm_AMB_PA: 0


In [387]:
malnutr_AMB_PA = AMB_PA['mna_resultats'].between(17, 23.5)
Percentatge_malnutr_AMB_PA = malnutr_AMB_PA.mean() * 100
num_observaciones_malnutr_AMB_PA = norm_AMB_PA.sum()

print(f"Percentatge de malnutr_AMB_PA: {Percentatge_malnutr_AMB_PA:.2f}%")
print(f"Nombre d'observacions en norm_AMB_PA: {num_observaciones_malnutr_AMB_PA}")

Percentatge de malnutr_AMB_PA: 0.00%
Nombre d'observacions en norm_AMB_PA: 0


In [388]:
malnut_AMB_PA = AMB_PA['mna_resultats'] < 17
Percentatge_malnut_AMB_PA = malnut_AMB_PA.mean() * 100
num_observaciones_malnut_AMB_PA = malnut_AMB_PA.sum()

print(f"Percentatge de malnut_AMB_PA: {Percentatge_malnut_AMB_PA:.2f}%")
print(f"Nombre d'observacions en malnut_AMB_PA: {num_observaciones_malnut_AMB_PA}")

Percentatge de malnut_AMB_PA: 23.53%
Nombre d'observacions en malnut_AMB_PA: 4


##### AMB_PA_MECVV

In [389]:
norm_AMB_PA_MECVV = AMB_PA_MECVV['mna_resultats'].between(24, 30)
Percentatge_norm_AMB_PA_MECVV = norm_AMB_PA_MECVV.mean() * 100
num_observaciones_norm_AMB_PA_MECVV = norm_AMB_PA_MECVV.sum()

print(f"Percentatge de norm_AMB_PA_MECVV: {Percentatge_norm_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en norm_AMB_PA_MECVV: {num_observaciones_norm_AMB_PA_MECVV}")

Percentatge de norm_AMB_PA_MECVV: 0.00%
Nombre d'observacions en norm_AMB_PA_MECVV: 0


In [390]:
malnutr_AMB_PA_MECVV = AMB_PA_MECVV['mna_resultats'].between(17, 23.5)
Percentatge_malnutr_AMB_PA_MECVV = malnutr_AMB_PA_MECVV.mean() * 100
num_observaciones_malnutr_AMB_PA_MECVV = norm_AMB_PA_MECVV.sum()

print(f"Percentatge de malnutr_AMB_PA_MECVV: {Percentatge_malnutr_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en norm_AMB_PA_MECVV: {num_observaciones_malnutr_AMB_PA_MECVV}")

Percentatge de malnutr_AMB_PA_MECVV: 0.00%
Nombre d'observacions en norm_AMB_PA_MECVV: 0


In [391]:
malnut_AMB_PA_MECVV = AMB_PA_MECVV['mna_resultats'] < 17
Percentatge_malnut_AMB_PA_MECVV = malnut_AMB_PA_MECVV.mean() * 100
num_observaciones_malnut_AMB_PA_MECVV = malnut_AMB_PA_MECVV.sum()

print(f"Percentatge de malnut_AMB_PA_MECVV: {Percentatge_malnut_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en malnut_AMB_PA_MECVV: {num_observaciones_malnut_AMB_PA_MECVV}")

Percentatge de malnut_AMB_PA_MECVV: 25.00%
Nombre d'observacions en malnut_AMB_PA_MECVV: 2


##### SENSE_PA

In [392]:
norm_SENSE_PA = SENSE_PA['mna_resultats'].between(24, 30)
Percentatge_norm_SENSE_PA = norm_SENSE_PA.mean() * 100
num_observaciones_norm_SENSE_PA = norm_SENSE_PA.sum()

print(f"Percentatge de norm_SENSE_PA: {Percentatge_norm_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en norm_SENSE_PA: {num_observaciones_norm_SENSE_PA}")

Percentatge de norm_SENSE_PA: 0.00%
Nombre d'observacions en norm_SENSE_PA: 0


In [393]:
malnutr_SENSE_PA = SENSE_PA['mna_resultats'].between(17, 23.5)
Percentatge_malnutr_SENSE_PA = malnutr_SENSE_PA.mean() * 100
num_observaciones_malnutr_SENSE_PA = malnutr_SENSE_PA.sum()

print(f"Percentatge de malnutr_SENSE_PA: {Percentatge_malnutr_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en malnutr_SENSE_PA: {num_observaciones_malnutr_SENSE_PA}")

Percentatge de malnutr_SENSE_PA: 2.04%
Nombre d'observacions en malnutr_SENSE_PA: 1


In [394]:
malnut_SENSE_PA = SENSE_PA['mna_resultats'] < 17
Percentatge_malnut_SENSE_PA = malnut_SENSE_PA.mean() * 100
num_observaciones_malnut_SENSE_PA = malnut_SENSE_PA.sum()

print(f"Percentatge de malnut_SENSE_PA: {Percentatge_malnut_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en malnut_SENSE_PA: {num_observaciones_malnut_SENSE_PA}")

Percentatge de malnut_SENSE_PA: 18.37%
Nombre d'observacions en malnut_SENSE_PA: 9


#### Pvalor

##### Normal

In [395]:
t_stat_12, p_val_12 = mannwhitneyu(AMB_PA['mna_resultats'].between(24, 30), AMB_PA_MECVV['mna_resultats'].between(24, 30), alternative='two-sided')
print("P-valor entre norm_AMB_PA i norm_AMB_PA_MECVV:", p_val_12)

P-valor entre norm_AMB_PA i norm_AMB_PA_MECVV: 1.0


In [396]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['mna_resultats'].between(24, 30), SENSE_PA['mna_resultats'].between(24, 30), alternative='two-sided')
print("P-valor entre norm_AMB_PA i norm_SENSE_PA:", p_val_13)

P-valor entre norm_AMB_PA i norm_SENSE_PA: 1.0


In [397]:
t_stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['mna_resultats'].between(24, 30), SENSE_PA['mna_resultats'].between(24, 30), alternative='two-sided')
print("P-valor entre norm_AMB_PA_MECVV i norm_SENSE_PA:", p_val_23)

P-valor entre norm_AMB_PA_MECVV i norm_SENSE_PA: 1.0


##### Risc de malnutrició

In [398]:
t_stat_12, p_val_12 = mannwhitneyu(AMB_PA['mna_resultats'].between(17, 23.5), AMB_PA_MECVV['mna_resultats'].between(17, 23.5), alternative='two-sided')
print("P-valor entre malnutr_AMB_PA i malnutr_AMB_PA_MECVV:", p_val_12)

P-valor entre malnutr_AMB_PA i malnutr_AMB_PA_MECVV: 1.0


In [399]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['mna_resultats'].between(17, 23.5), SENSE_PA['mna_resultats'].between(17, 23.5), alternative='two-sided')
print("P-valor entre malnutr_AMB_PA i malnutr_SENSE_PA:", p_val_13)

P-valor entre malnutr_AMB_PA i malnutr_SENSE_PA: 0.5793276078637376


In [400]:
t_stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['mna_resultats'].between(17, 23.5), SENSE_PA['mna_resultats'].between(17, 23.5), alternative='two-sided')
print("P-valor entre malnutr_AMB_PA_MECVV i malnutr_SENSE_PA:", p_val_23)

P-valor entre malnutr_AMB_PA_MECVV i malnutr_SENSE_PA: 0.7236736098317631


##### Malnutrició

In [401]:
t_stat_12, p_val_12 = mannwhitneyu(AMB_PA['mna_resultats'] <17 , 
                                AMB_PA_MECVV['mna_resultats'] < 17, alternative='two-sided')
print("P-valor entre malnut_AMB_PA i norm_AMB_PA_MECVV:", p_val_12)

P-valor entre malnut_AMB_PA i norm_AMB_PA_MECVV: 0.9686157491081862


In [402]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['mna_resultats'] <17 , 
                                SENSE_PA['mna_resultats'] < 17, alternative='two-sided')
print("P-valor entre malnut_AMB_PA i malnut_SENSE_PA:", p_val_13)

P-valor entre malnut_AMB_PA i malnut_SENSE_PA: 0.6549000779898075


In [403]:
t_stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['mna_resultats'] <17 , 
                                SENSE_PA['mna_resultats'] < 17, alternative='two-sided')
print("P-valor entre malnut_AMB_PA_MECVV i malnut_SENSE_PA:", p_val_23)

P-valor entre malnut_AMB_PA_MECVV i malnut_SENSE_PA: 0.67443851227345


## Index de l'EMINA

### Mitjana

In [404]:
promedio_emina = AMB_PA['EMINA_sumatorios_comparados'].mean()
desviacion_estandar_emina = AMB_PA['EMINA_sumatorios_comparados'].std()

resultado = f"{promedio_emina:.2f} ± {desviacion_estandar_emina:.2f}"
print("Mitjana ± Desviació Estàndard de l'emina:", resultado)

Mitjana ± Desviació Estàndard de l'emina: 7.94 ± 3.21


In [405]:
promedio_emina = AMB_PA_MECVV['EMINA_sumatorios_comparados'].mean()
desviacion_estandar_emina = AMB_PA_MECVV['EMINA_sumatorios_comparados'].std()

resultado = f"{promedio_emina:.2f} ± {desviacion_estandar_emina:.2f}"
print("Mitjana ± Desviació Estàndard de l'emina:", resultado)

Mitjana ± Desviació Estàndard de l'emina: 8.50 ± 1.51


In [406]:
promedio_emina = SENSE_PA['EMINA_sumatorios_comparados'].mean()
desviacion_estandar_emina = SENSE_PA['EMINA_sumatorios_comparados'].std()

resultado = f"{promedio_emina:.2f} ± {desviacion_estandar_emina:.2f}"
print("Mitjana ± Desviació Estàndard de l'emina:", resultado)

Mitjana ± Desviació Estàndard de l'emina: 5.64 ± 4.08


#### Pvalor

##### Normalitat

In [407]:
stat, p_value = shapiro(AMB_PA['EMINA_sumatorios_comparados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9770
Valor p = 0.9248
El nombre de files segueix una distribució normal.


In [408]:
stat, p_value = shapiro(AMB_PA_MECVV['EMINA_sumatorios_comparados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9183
Valor p = 0.4158
El nombre de files segueix una distribució normal.


In [409]:
stat, p_value = shapiro(SENSE_PA['EMINA_sumatorios_comparados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [410]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['EMINA_sumatorios_comparados'], AMB_PA_MECVV['EMINA_sumatorios_comparados'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 0.5598635957447045


In [411]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['EMINA_sumatorios_comparados'], SENSE_PA['EMINA_sumatorios_comparados'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [412]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['EMINA_sumatorios_comparados'], SENSE_PA['EMINA_sumatorios_comparados'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Valors EMINA

##### AMB_PA

In [413]:
lowr_AMB_PA = AMB_PA['EMINA_sumatorios_comparados'].between(0, 5)
Percentatge_lowr_AMB_PA = lowr_AMB_PA.mean() * 100
num_observaciones_lowr_AMB_PA = lowr_AMB_PA.sum()

print(f"Percentatge de lowr_AMB_PA: {Percentatge_lowr_AMB_PA:.2f}%")
print(f"Nombre d'observacions en lowr_AMB_PA: {num_observaciones_lowr_AMB_PA}")

Percentatge de lowr_AMB_PA: 17.65%
Nombre d'observacions en lowr_AMB_PA: 3


In [414]:
modr_AMB_PA = AMB_PA['EMINA_sumatorios_comparados'].between(6, 10)
Percentatge_modr_AMB_PA = modr_AMB_PA.mean() * 100
num_observaciones_modr_AMB_PA = modr_AMB_PA.sum()

print(f"Percentatge de modr_AMB_PA: {Percentatge_modr_AMB_PA:.2f}%")
print(f"Nombre d'observacions en modr_AMB_PA: {num_observaciones_modr_AMB_PA}")

Percentatge de modr_AMB_PA: 64.71%
Nombre d'observacions en modr_AMB_PA: 11


In [415]:
highr_AMB_PA = AMB_PA['EMINA_sumatorios_comparados'].between(11, 15)
Percentatge_highr_AMB_PA = highr_AMB_PA.mean() * 100
num_observaciones_highr_AMB_PA = highr_AMB_PA.sum()

print(f"Percentatge de highr_AMB_PA: {Percentatge_highr_AMB_PA:.2f}%")
print(f"Nombre d'observacions en highr_AMB_PA: {num_observaciones_highr_AMB_PA}")

Percentatge de highr_AMB_PA: 17.65%
Nombre d'observacions en highr_AMB_PA: 3


##### AMB_PA_MECVV

In [416]:
lowr_AMB_PA_MECVV = AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(0, 5)
Percentatge_lowr_AMB_PA_MECVV = lowr_AMB_PA_MECVV.mean() * 100
num_observaciones_lowr_AMB_PA_MECVV = lowr_AMB_PA_MECVV.sum()

print(f"Percentatge de lowr_AMB_PA_MECVV: {Percentatge_lowr_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en lowr_AMB_PA_MECVV: {num_observaciones_lowr_AMB_PA_MECVV}")

Percentatge de lowr_AMB_PA_MECVV: 0.00%
Nombre d'observacions en lowr_AMB_PA_MECVV: 0


In [417]:
modr_AMB_PA_MECVV = AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(6, 10)
Percentatge_modr_AMB_PA_MECVV = modr_AMB_PA_MECVV.mean() * 100
num_observaciones_modr_AMB_PA_MECVV = modr_AMB_PA_MECVV.sum()

print(f"Percentatge de modr_AMB_PA_MECVV: {Percentatge_modr_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en modr_AMB_PA_MECVV: {num_observaciones_modr_AMB_PA_MECVV}")

Percentatge de modr_AMB_PA_MECVV: 87.50%
Nombre d'observacions en modr_AMB_PA_MECVV: 7


In [418]:
highr_AMB_PA_MECVV = AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(11, 15)
Percentatge_highr_AMB_PA_MECVV = highr_AMB_PA_MECVV.mean() * 100
num_observaciones_highr_AMB_PA_MECVV = highr_AMB_PA_MECVV.sum()

print(f"Percentatge de highr_AMB_PA_MECVV: {Percentatge_highr_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en highr_AMB_PA_MECVV: {num_observaciones_highr_AMB_PA_MECVV}")

Percentatge de highr_AMB_PA_MECVV: 12.50%
Nombre d'observacions en highr_AMB_PA_MECVV: 1


##### SENSE_PA

In [419]:
lowr_SENSE_PA = SENSE_PA['EMINA_sumatorios_comparados'].between(0, 5)
Percentatge_lowr_SENSE_PA = lowr_SENSE_PA.mean() * 100
num_observaciones_lowr_SENSE_PA = lowr_SENSE_PA.sum()

print(f"Percentatge de lowr_SENSE_PA: {Percentatge_lowr_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en lowr_SENSE_PA: {num_observaciones_lowr_SENSE_PA}")

Percentatge de lowr_SENSE_PA: 44.90%
Nombre d'observacions en lowr_SENSE_PA: 22


In [420]:
modr_SENSE_PA = SENSE_PA['EMINA_sumatorios_comparados'].between(6, 10)
Percentatge_modr_SENSE_PA = modr_SENSE_PA.mean() * 100
num_observaciones_modr_SENSE_PA = modr_SENSE_PA.sum()

print(f"Percentatge de modr_SENSE_PA: {Percentatge_modr_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en modr_SENSE_PA: {num_observaciones_modr_SENSE_PA}")

Percentatge de modr_SENSE_PA: 40.82%
Nombre d'observacions en modr_SENSE_PA: 20


In [421]:
highr_SENSE_PA = SENSE_PA['EMINA_sumatorios_comparados'].between(11, 15)
Percentatge_highr_SENSE_PA = highr_SENSE_PA.mean() * 100
num_observaciones_highr_SENSE_PA = highr_SENSE_PA.sum()

print(f"Percentatge de highr_SENSE_PA: {Percentatge_highr_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en highr_SENSE_PA: {num_observaciones_highr_SENSE_PA}")

Percentatge de highr_SENSE_PA: 10.20%
Nombre d'observacions en highr_SENSE_PA: 5


#### Pvalor 

##### Risc baix

In [422]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['EMINA_sumatorios_comparados'].between(0,5), AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(0,5), equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 0.08262239198015026


In [423]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['EMINA_sumatorios_comparados'].between(0,5), SENSE_PA['EMINA_sumatorios_comparados'].between(0,5), equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: 0.028470032037651503


In [424]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(0,5), SENSE_PA['EMINA_sumatorios_comparados'].between(0,5), equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: 1.0253507863008461e-07


##### Risc moderat

In [425]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['EMINA_sumatorios_comparados'].between(6,10), AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(6,10), equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 0.20327534354395935


In [426]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['EMINA_sumatorios_comparados'].between(6,10), SENSE_PA['EMINA_sumatorios_comparados'].between(6,10), equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: 0.09654829012396034


In [427]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(6,10), SENSE_PA['EMINA_sumatorios_comparados'].between(6,10), equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.0069446206793742


##### Risc alt

In [428]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['EMINA_sumatorios_comparados'].between(11,15), AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(11,15), equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 0.7477825496543375


In [429]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['EMINA_sumatorios_comparados'].between(11,15), SENSE_PA['EMINA_sumatorios_comparados'].between(11,15), equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: 0.48486212369214565


In [430]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(11,15), SENSE_PA['EMINA_sumatorios_comparados'].between(11,15), equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.8662716025551493


## Index de la canadenca 

### Mitjana

In [431]:
promedio_canad = AMB_PA['Canadenca_sumatorios_comparados'].mean()
desviacion_estandar_canad = AMB_PA['Canadenca_sumatorios_comparados'].std()

resultado = f"{promedio_canad:.2f} ± {desviacion_estandar_canad:.2f}"
print("Mitjana ± Desviació Estàndard de la canadenca:", resultado)

Mitjana ± Desviació Estàndard de la canadenca: 7.00 ± nan


In [432]:
promedio_canad = AMB_PA_MECVV['Canadenca_sumatorios_comparados'].mean()
desviacion_estandar_canad = AMB_PA_MECVV['Canadenca_sumatorios_comparados'].std()

resultado = f"{promedio_canad:.2f} ± {desviacion_estandar_canad:.2f}"
print("Mitjana ± Desviació Estàndard de la canadenca:", resultado)

Mitjana ± Desviació Estàndard de la canadenca: nan ± nan


In [433]:
promedio_canad = SENSE_PA['Canadenca_sumatorios_comparados'].mean()
desviacion_estandar_canad = SENSE_PA['Canadenca_sumatorios_comparados'].std()

resultado = f"{promedio_canad:.2f} ± {desviacion_estandar_canad:.2f}"
print("Mitjana ± Desviació Estàndard de la canadenca:", resultado)

Mitjana ± Desviació Estàndard de la canadenca: 8.00 ± 1.41


#### Pvalor

##### Normalitat

In [434]:
stat, p_value = shapiro(AMB_PA['Canadenca_sumatorios_comparados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [435]:
stat, p_value = shapiro(AMB_PA_MECVV['Canadenca_sumatorios_comparados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [436]:
stat, p_value = shapiro(SENSE_PA['Canadenca_sumatorios_comparados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [437]:
t_stat_12, p_val_12 = mannwhitneyu(AMB_PA['Canadenca_sumatorios_comparados'], AMB_PA_MECVV['Canadenca_sumatorios_comparados'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [438]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['Canadenca_sumatorios_comparados'], SENSE_PA['Canadenca_sumatorios_comparados'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [439]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Canadenca_sumatorios_comparados'], SENSE_PA['Canadenca_sumatorios_comparados'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Valors canadenca

##### AMB_PA 

In [440]:
mild_AMB_PA = AMB_PA['Canadenca_sumatorios_comparados'].between(10.5, 11.5)
Percentatge_mild_AMB_PA = mild_AMB_PA.mean() * 100
num_observaciones_mild_AMB_PA = mild_AMB_PA.sum()

print(f"Percentatge de mild_AMB_PA: {Percentatge_mild_AMB_PA:.2f}%")
print(f"Nombre d'observacions en mild_AMB_PA: {num_observaciones_mild_AMB_PA}")

Percentatge de mild_AMB_PA: 0.00%
Nombre d'observacions en mild_AMB_PA: 0


In [441]:
modneu_AMB_PA = AMB_PA['Canadenca_sumatorios_comparados'].between(5,10)
Percentatge_modneu_AMB_PA = modneu_AMB_PA.mean() * 100
num_observaciones_modneu_AMB_PA = modneu_AMB_PA.sum()

print(f"Percentatge de modneu_AMB_PA: {Percentatge_modneu_AMB_PA:.2f}%")
print(f"Nombre d'observacions en modneu_AMB_PA: {num_observaciones_modneu_AMB_PA}")

Percentatge de modneu_AMB_PA: 5.88%
Nombre d'observacions en modneu_AMB_PA: 1


In [442]:
sevneu_AMB_PA = AMB_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5)
Percentatge_sevneu_AMB_PA = sevneu_AMB_PA.mean() * 100
num_observaciones_sevneu_AMB_PA = sevneu_AMB_PA.sum()

print(f"Percentatge de sevneu_AMB_PA: {Percentatge_sevneu_AMB_PA:.2f}%")
print(f"Nombre d'observacions en sevneu_AMB_PA: {num_observaciones_sevneu_AMB_PA}")

Percentatge de sevneu_AMB_PA: 0.00%
Nombre d'observacions en sevneu_AMB_PA: 0


##### AMB_PA_MECVV

In [443]:
mild_AMB_PA_MECVV = AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(10.5, 11.5)
Percentatge_mild_AMB_PA_MECVV = mild_AMB_PA_MECVV.mean() * 100
num_observaciones_mild_AMB_PA_MECVV = mild_AMB_PA_MECVV.sum()

print(f"Percentatge de mild_AMB_PA_MECVV: {Percentatge_mild_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en mild_AMB_PA_MECVV: {num_observaciones_mild_AMB_PA_MECVV}")

Percentatge de mild_AMB_PA_MECVV: 0.00%
Nombre d'observacions en mild_AMB_PA_MECVV: 0


In [444]:
modneu_AMB_PA_MECVV = AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(5,10)
Percentatge_modneu_AMB_PA_MECVV = modneu_AMB_PA_MECVV.mean() * 100
num_observaciones_modneu_AMB_PA_MECVV = modneu_AMB_PA_MECVV.sum()

print(f"Percentatge de modneu_AMB_PA_MECVV: {Percentatge_modneu_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en modneu_AMB_PA_MECVV: {num_observaciones_modneu_AMB_PA_MECVV}")

Percentatge de modneu_AMB_PA_MECVV: 0.00%
Nombre d'observacions en modneu_AMB_PA_MECVV: 0


In [445]:
sevneu_AMB_PA_MECVV = AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(1.5, 4.5)
Percentatge_sevneu_AMB_PA_MECVV = sevneu_AMB_PA_MECVV.mean() * 100
num_observaciones_sevneu_AMB_PA_MECVV = sevneu_AMB_PA_MECVV.sum()

print(f"Percentatge de sevneu_AMB_PA_MECVV: {Percentatge_sevneu_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en sevneu_AMB_PA_MECVV: {num_observaciones_sevneu_AMB_PA_MECVV}")

Percentatge de sevneu_AMB_PA_MECVV: 0.00%
Nombre d'observacions en sevneu_AMB_PA_MECVV: 0


##### SENSE_PA

In [446]:
mild_SENSE_PA = SENSE_PA['Canadenca_sumatorios_comparados'].between(10.5, 11.5)
Percentatge_mild_SENSE_PA = mild_SENSE_PA.mean() * 100
num_observaciones_mild_SENSE_PA = mild_SENSE_PA.sum()

print(f"Percentatge de mild_SENSE_PA: {Percentatge_mild_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en mild_SENSE_PA: {num_observaciones_mild_SENSE_PA}")

Percentatge de mild_SENSE_PA: 0.00%
Nombre d'observacions en mild_SENSE_PA: 0


In [447]:
modneu_SENSE_PA = SENSE_PA['Canadenca_sumatorios_comparados'].between(5,10)
Percentatge_modneu_SENSE_PA = modneu_SENSE_PA.mean() * 100
num_observaciones_modneu_SENSE_PA = modneu_SENSE_PA.sum()

print(f"Percentatge de modneu_SENSE_PA: {Percentatge_modneu_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en modneu_SENSE_PA: {num_observaciones_modneu_SENSE_PA}")

Percentatge de modneu_SENSE_PA: 4.08%
Nombre d'observacions en modneu_SENSE_PA: 2


In [448]:
sevneu_SENSE_PA = SENSE_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5)
Percentatge_sevneu_SENSE_PA = sevneu_SENSE_PA.mean() * 100
num_observaciones_sevneu_SENSE_PA = sevneu_SENSE_PA.sum()

print(f"Percentatge de sevneu_SENSE_PA: {Percentatge_sevneu_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en sevneu_SENSE_PA: {num_observaciones_sevneu_SENSE_PA}")

Percentatge de sevneu_SENSE_PA: 0.00%
Nombre d'observacions en sevneu_SENSE_PA: 0


#### Pvalor

##### Dèficit neurològic lleuger (mild)

In [449]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['Canadenca_sumatorios_comparados'].between(10.5,11.5), AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(10.5,11.5), equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [450]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['Canadenca_sumatorios_comparados'].between(10.5,11.5), SENSE_PA['Canadenca_sumatorios_comparados'].between(10.5,11.5), equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [451]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(10.5,11.5), SENSE_PA['Canadenca_sumatorios_comparados'].between(10.5,11.5), equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


##### Dèficit neurològic moderat

In [452]:
t_stat_12, p_val_12 = mannwhitneyu(AMB_PA['Canadenca_sumatorios_comparados'].between(5,10), AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(5, 10), alternative='less')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 0.7798668588314288


In [453]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['Canadenca_sumatorios_comparados'].between(5,10), SENSE_PA['Canadenca_sumatorios_comparados'].between(5, 10), alternative='less')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: 0.6274490868971419


In [454]:
t_stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(5,10), SENSE_PA['Canadenca_sumatorios_comparados'].between(5, 10), alternative='less')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.2944045932704268


##### Dèficit neurològic greu (sever)

In [455]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['Canadenca_sumatorios_comparados'].between(1.5,4.5), AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(1.5,4.5), equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [456]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5), SENSE_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5), equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [457]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(1.5,4.5), SENSE_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5), equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


## Infeccions respiratòries

##### AMB_PA 

In [458]:
mainresp_AMB_PA= (AMB_PA['Main_respiratory_infections_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de infeccions respiratòries principals : {mainresp_AMB_PA:.2f}%")

Percentatge de infeccions respiratòries principals : 41.18%


In [459]:
LRTI_AMB_PA= (AMB_PA['LRTI_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de LRTI: {LRTI_AMB_PA:.2f}%")

Percentatge de LRTI: 17.65%


In [460]:
COPD_AMB_PA= (AMB_PA['COPD_exacerbations_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de exacerbacions de COPD: {COPD_AMB_PA:.2f}%")

Percentatge de exacerbacions de COPD: 29.41%


In [461]:
fibro_AMB_PA= (AMB_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de fibrosi pulmonar i fibrotòrax: {fibro_AMB_PA:.2f}%")

Percentatge de fibrosi pulmonar i fibrotòrax: 0.00%


##### AMB_PA_MECVV

In [462]:
mainresp_AMB_PA_MECVV= (AMB_PA_MECVV['Main_respiratory_infections_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de infeccions respiratòries principals : {mainresp_AMB_PA_MECVV:.2f}%")

Percentatge de infeccions respiratòries principals : 50.00%


In [463]:
LRTI_AMB_PA_MECVV= (AMB_PA_MECVV['LRTI_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de LRTI: {LRTI_AMB_PA_MECVV:.2f}%")

Percentatge de LRTI: 12.50%


In [464]:
COPD_AMB_PA_MECVV = (AMB_PA_MECVV['COPD_exacerbations_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de exacerbacions de COPD: {COPD_AMB_PA_MECVV:.2f}%")

Percentatge de exacerbacions de COPD: 50.00%


In [465]:
fibro_AMB_PA_MECVV = (AMB_PA_MECVV['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de fibrosi pulmonar i fibrotòrax: {fibro_AMB_PA_MECVV:.2f}%")

Percentatge de fibrosi pulmonar i fibrotòrax: 0.00%


##### SENSE_PA

In [466]:
mainresp_SENSE_PA= (SENSE_PA['Main_respiratory_infections_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de infeccions respiratòries principals : {mainresp_SENSE_PA:.2f}%")

Percentatge de infeccions respiratòries principals : 42.86%


In [467]:
LRTI_SENSE_PA= (SENSE_PA['LRTI_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de LRTI: {LRTI_SENSE_PA:.2f}%")

Percentatge de LRTI: 22.45%


In [468]:
COPD_SENSE_PA= (SENSE_PA['COPD_exacerbations_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de exacerbacions de COPD: {COPD_SENSE_PA:.2f}%")

Percentatge de exacerbacions de COPD: 32.65%


In [469]:
fibro_SENSE_PA= (SENSE_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de fibrosi pulmonar i fibrotòrax: {fibro_SENSE_PA:.2f}%")

Percentatge de fibrosi pulmonar i fibrotòrax: 0.00%


#### Pvalor

##### Normalitat

In [470]:
stat, p_value = shapiro(AMB_PA['Main_respiratory_infections_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.6317
Valor p = 0.00002144
El nombre de files no segueix una distribució normal.


In [471]:
stat, p_value = shapiro(AMB_PA['LRTI_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.4702
Valor p = 0.00000073
El nombre de files no segueix una distribució normal.


In [472]:
stat, p_value = shapiro(AMB_PA['COPD_exacerbations_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.5785
Valor p = 0.00000650
El nombre de files no segueix una distribució normal.


In [473]:
stat, p_value = shapiro(AMB_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 1.0000
Valor p = 1.00000000
El nombre de files segueix una distribució normal.


C:\Users\maria\PycharmProjects\practicum\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [474]:
stat, p_value = shapiro(AMB_PA_MECVV['Main_respiratory_infections_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.6647
Valor p = 0.00089170
El nombre de files no segueix una distribució normal.


In [475]:
stat, p_value = shapiro(AMB_PA_MECVV['LRTI_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.4184
Valor p = 0.00000105
El nombre de files no segueix una distribució normal.


In [476]:
stat, p_value = shapiro(AMB_PA_MECVV['COPD_exacerbations_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.6647
Valor p = 0.00089170
El nombre de files no segueix una distribució normal.


In [477]:
stat, p_value = shapiro(AMB_PA_MECVV['Pulmonary_fibrosis_fibrotorax_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 1.0000
Valor p = 1.00000000
El nombre de files segueix una distribució normal.


In [478]:
stat, p_value = shapiro(SENSE_PA['Main_respiratory_infections_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.6294
Valor p = 0.00000000
El nombre de files no segueix una distribució normal.


In [479]:
stat, p_value = shapiro(SENSE_PA['LRTI_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.5158
Valor p = 0.00000000
El nombre de files no segueix una distribució normal.


In [480]:
stat, p_value = shapiro(SENSE_PA['COPD_exacerbations_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.5916
Valor p = 0.00000000
El nombre de files no segueix una distribució normal.


In [481]:
stat, p_value = shapiro(SENSE_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 1.0000
Valor p = 1.00000000
El nombre de files segueix una distribució normal.


##### Infeccions respiratòries principals 

In [482]:
stat_12, p_val_12 = mannwhitneyu(AMB_PA['Main_respiratory_infections_diagnosticada'] == 1.0, AMB_PA_MECVV['Main_respiratory_infections_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 0.7096213402025467


In [483]:
stat_13, p_val_13 = mannwhitneyu(AMB_PA['Main_respiratory_infections_diagnosticada'] == 1.0, SENSE_PA['Main_respiratory_infections_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: 0.9113557832989896


In [484]:
stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['Main_respiratory_infections_diagnosticada'] == 1.0, SENSE_PA['Main_respiratory_infections_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.7182469617733183


##### LRTI

In [485]:
stat_12, p_val_12 = mannwhitneyu(AMB_PA['LRTI_diagnosticada'] == 1.0, AMB_PA_MECVV['LRTI_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 0.7833089019485086


In [486]:
stat_13, p_val_13 = mannwhitneyu(AMB_PA['LRTI_diagnosticada'] == 1.0, SENSE_PA['LRTI_diagnosticada'] == 1.0,alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: 0.6863825106164784


In [487]:
stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['LRTI_diagnosticada'] == 1.0, SENSE_PA['LRTI_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.5365291663168164


##### Exacerbacions de COPD

In [488]:
stat_12, p_val_12 = mannwhitneyu(AMB_PA['COPD_exacerbations_diagnosticada'] == 1.0, AMB_PA_MECVV['COPD_exacerbations_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 0.34456236532569795


In [489]:
stat_13, p_val_13 = mannwhitneyu(AMB_PA['COPD_exacerbations_diagnosticada'] == 1.0, SENSE_PA['COPD_exacerbations_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: 0.8132303236327403


In [490]:
stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['COPD_exacerbations_diagnosticada'] == 1.0, SENSE_PA['COPD_exacerbations_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.3518976515408023


##### Fibrosi pulmonar i fibrotòrax

In [491]:
stat_12, p_val_12 = mannwhitneyu(AMB_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, AMB_PA_MECVV['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 1.0


In [492]:
stat_13, p_val_13 = mannwhitneyu(AMB_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, SENSE_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: 1.0


In [493]:
stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, SENSE_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: 1.0


## Resultats de laboratori

Les columnes generades a partir d'aquests resultats contenen valors tipus objecte, per tant, primer els transformaré a tipus numèric per poder realitzar les operacions que neccessitaré fer.

### Albúmina

In [494]:
AMB_PA.loc[:, 'albumina'] = pd.to_numeric(AMB_PA['albumina'], errors='coerce')

promedio_alb = AMB_PA['albumina'].mean()
desviacion_estandar_alb = AMB_PA['albumina'].std()

resultado = f"{promedio_alb:.2f} ± {desviacion_estandar_alb:.2f}"
print("Mitjana ± Desviació Estàndard de la albumina:", resultado)

Mitjana ± Desviació Estàndard de la albumina: 3.34 ± 0.38


In [495]:
AMB_PA_MECVV.loc[:, 'albumina'] = pd.to_numeric(AMB_PA_MECVV['albumina'], errors='coerce')

promedio_alb = AMB_PA_MECVV['albumina'].mean()
desviacion_estandar_alb = AMB_PA_MECVV['albumina'].std()

resultado = f"{promedio_alb:.2f} ± {desviacion_estandar_alb:.2f}"
print("Mitjana ± Desviació Estàndard de la albumina:", resultado)

Mitjana ± Desviació Estàndard de la albumina: 3.33 ± 0.38


In [496]:
SENSE_PA.loc[:, 'albumina'] = pd.to_numeric(SENSE_PA['albumina'], errors='coerce')

promedio_alb = SENSE_PA['albumina'].mean()
desviacion_estandar_alb = SENSE_PA['albumina'].std()

resultado = f"{promedio_alb:.2f} ± {desviacion_estandar_alb:.2f}"
print("Mitjana ± Desviació Estàndard de la albumina:", resultado)

Mitjana ± Desviació Estàndard de la albumina: 3.58 ± 0.64


#### Pvalor

##### Normalitat

In [497]:
albpa = AMB_PA['albumina'].dropna()
stat, p_value = shapiro(albpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9311
Valor p = 0.56068160
El nombre de files segueix una distribució normal.


In [498]:
albAMBpamecvv = AMB_PA_MECVV['albumina'].dropna()
stat, p_value = shapiro(albAMBpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8547
Valor p = 0.25296965
El nombre de files segueix una distribució normal.


In [499]:
albsensepa= SENSE_PA['albumina'].dropna()
stat, p_value = shapiro(albsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9778
Valor p = 0.92442182
El nombre de files segueix una distribució normal.


In [500]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['albumina'], AMB_PA_MECVV['albumina'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [501]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['albumina'], SENSE_PA['albumina'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [502]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['albumina'], SENSE_PA['albumina'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Proteïnes totals

In [503]:
AMB_PA.loc[:, 'proteinas totales'] = pd.to_numeric(AMB_PA['proteinas totales'], errors='coerce')

promedio_prot = AMB_PA['proteinas totales'].mean()
desviacion_estandar_prot = AMB_PA['proteinas totales'].std()

resultado = f"{promedio_prot:.2f} ± {desviacion_estandar_prot:.2f}"
print("Mitjana ± Desviació Estàndard de las proteinas totales:", resultado)

Mitjana ± Desviació Estàndard de las proteinas totales: 6.03 ± 0.73


In [504]:
AMB_PA_MECVV.loc[:, 'proteinas totales'] = pd.to_numeric(AMB_PA_MECVV['proteinas totales'], errors='coerce')

promedio_prot = AMB_PA_MECVV['proteinas totales'].mean()
desviacion_estandar_prot = AMB_PA_MECVV['proteinas totales'].std()

resultado = f"{promedio_prot:.2f} ± {desviacion_estandar_prot:.2f}"
print("Mitjana ± Desviació Estàndard de las proteinas totales:", resultado)

Mitjana ± Desviació Estàndard de las proteinas totales: 6.00 ± 0.53


In [505]:
SENSE_PA.loc[:, 'proteinas totales'] = pd.to_numeric(SENSE_PA['proteinas totales'], errors='coerce')

promedio_prot = SENSE_PA['proteinas totales'].mean()
desviacion_estandar_prot = SENSE_PA['proteinas totales'].std()

resultado = f"{promedio_prot:.2f} ± {desviacion_estandar_prot:.2f}"
print("Mitjana ± Desviació Estàndard de las proteinas totales:", resultado)

Mitjana ± Desviació Estàndard de las proteinas totales: 6.12 ± 0.72


#### Pvalor

##### Normalitat

In [506]:
protAMBpa=AMB_PA['proteinas totales'].dropna()
stat, p_value = shapiro(protAMBpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9780
Valor p = 0.95346065
El nombre de files segueix una distribució normal.


In [507]:
protAMBpamecvv=AMB_PA_MECVV['proteinas totales'].dropna()
stat, p_value = shapiro(protAMBpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8929
Valor p = 0.36311315
El nombre de files segueix una distribució normal.


In [508]:
protsensepa=SENSE_PA['proteinas totales'].dropna()
stat, p_value = shapiro(protsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9753
Valor p = 0.86021925
El nombre de files segueix una distribució normal.


In [509]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['proteinas totales'], AMB_PA_MECVV['proteinas totales'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [510]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['proteinas totales'], SENSE_PA['proteinas totales'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [511]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['proteinas totales'], SENSE_PA['proteinas totales'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Hemoglobina

In [512]:
AMB_PA.loc[:, 'Hb'] = pd.to_numeric(AMB_PA['Hb'], errors='coerce')

promedio_hb = AMB_PA['Hb'].mean()
desviacion_estandar_hb = AMB_PA['Hb'].std()

resultado = f"{promedio_hb:.2f} ± {desviacion_estandar_hb:.2f}"
print("Mitjana ± Desviació Estàndard de la hemoglobina:", resultado)

Mitjana ± Desviació Estàndard de la hemoglobina: 12.34 ± 1.77


In [513]:
AMB_PA_MECVV.loc[:, 'Hb'] = pd.to_numeric(AMB_PA_MECVV['Hb'], errors='coerce')

promedio_hb = AMB_PA_MECVV['Hb'].mean()
desviacion_estandar_hb = AMB_PA_MECVV['Hb'].std()

resultado = f"{promedio_hb:.2f} ± {desviacion_estandar_hb:.2f}"
print("Mitjana ± Desviació Estàndard de la hemoglobina:", resultado)

Mitjana ± Desviació Estàndard de la hemoglobina: 11.77 ± 2.58


In [514]:
SENSE_PA.loc[:, 'Hb'] = pd.to_numeric(SENSE_PA['Hb'], errors='coerce')

promedio_hb = SENSE_PA['Hb'].mean()
desviacion_estandar_hb = SENSE_PA['Hb'].std()

resultado = f"{promedio_hb:.2f} ± {desviacion_estandar_hb:.2f}"
print("Mitjana ± Desviació Estàndard de la hemoglobina:", resultado)

Mitjana ± Desviació Estàndard de la hemoglobina: 13.22 ± 2.16


##### Pvalor

##### Normalitat

In [515]:
hbAMBpa=AMB_PA['Hb'].dropna()
stat, p_value = shapiro(hbAMBpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9304
Valor p = 0.48501101
El nombre de files segueix una distribució normal.


In [516]:
hbAMBpamecvv=AMB_PA_MECVV['Hb'].dropna()
stat, p_value = shapiro(hbAMBpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8288
Valor p = 0.18525796
El nombre de files segueix una distribució normal.


In [517]:
hbsensepa=SENSE_PA['Hb'].dropna()
stat, p_value = shapiro(hbsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9551
Valor p = 0.37239632
El nombre de files segueix una distribució normal.


In [518]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['Hb'], AMB_PA_MECVV['Hb'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [519]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['Hb'], SENSE_PA['Hb'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [520]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Hb'], SENSE_PA['Hb'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Colesterol 

In [521]:
AMB_PA.loc[:, 'colesterol total'] = pd.to_numeric(AMB_PA['colesterol total'], errors='coerce')

promedio_col = AMB_PA['colesterol total'].mean()
desviacion_estandar_col = AMB_PA['colesterol total'].std()

resultado = f"{promedio_col:.2f} ± {desviacion_estandar_col:.2f}"
print("Mitjana ± Desviació Estàndard del colesterol total:", resultado)

Mitjana ± Desviació Estàndard del colesterol total: 132.10 ± 34.83


In [522]:
AMB_PA_MECVV.loc[:, 'colesterol total'] = pd.to_numeric(AMB_PA_MECVV['colesterol total'], errors='coerce')

promedio_col = AMB_PA_MECVV['colesterol total'].mean()
desviacion_estandar_col = AMB_PA_MECVV['colesterol total'].std()

resultado = f"{promedio_col:.2f} ± {desviacion_estandar_col:.2f}"
print("Mitjana ± Desviació Estàndard del colesterol total:", resultado)

Mitjana ± Desviació Estàndard del colesterol total: 151.47 ± 33.55


In [523]:
SENSE_PA.loc[:, 'colesterol total'] = pd.to_numeric(SENSE_PA['colesterol total'], errors='coerce')

promedio_col = SENSE_PA['colesterol total'].mean()
desviacion_estandar_col = SENSE_PA['colesterol total'].std()

resultado = f"{promedio_col:.2f} ± {desviacion_estandar_col:.2f}"
print("Mitjana ± Desviació Estàndard del colesterol total:", resultado)

Mitjana ± Desviació Estàndard del colesterol total: 145.27 ± 40.45


#### Pvalor

##### Normalitat 



In [524]:
colAMBpa=AMB_PA['colesterol total'].dropna()
stat, p_value = shapiro(colAMBpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9887
Valor p = 0.99048226
El nombre de files segueix una distribució normal.


In [525]:
colAMBpamecvv=AMB_PA_MECVV['colesterol total'].dropna()
stat, p_value = shapiro(colAMBpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9941
Valor p = 0.85359605
El nombre de files segueix una distribució normal.


In [526]:
colsensepa=SENSE_PA['colesterol total'].dropna()
stat, p_value = shapiro(colsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9725
Valor p = 0.84373822
El nombre de files segueix una distribució normal.


In [527]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['colesterol total'], AMB_PA_MECVV['colesterol total'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [528]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['colesterol total'], SENSE_PA['colesterol total'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [529]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['colesterol total'], SENSE_PA['colesterol total'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Leucòcits

In [530]:
AMB_PA.loc[:, 'leucos'] = pd.to_numeric(AMB_PA['leucos'], errors='coerce')

promedio_leu = AMB_PA['leucos'].mean()
desviacion_estandar_leu = AMB_PA['leucos'].std()

resultado = f"{promedio_leu:.2f} ± {desviacion_estandar_leu:.2f}"
print("Mitjana ± Desviació Estàndard dels leucòcits:", resultado)

Mitjana ± Desviació Estàndard dels leucòcits: 12.69 ± 4.88


In [531]:
AMB_PA_MECVV.loc[:, 'leucos'] = pd.to_numeric(AMB_PA_MECVV['leucos'], errors='coerce')

promedio_leu = AMB_PA_MECVV['leucos'].mean()
desviacion_estandar_leu = AMB_PA_MECVV['leucos'].std()

resultado = f"{promedio_leu:.2f} ± {desviacion_estandar_leu:.2f}"
print("Mitjana ± Desviació Estàndard dels leucòcits:", resultado)

Mitjana ± Desviació Estàndard dels leucòcits: 8.83 ± 0.70


In [532]:
SENSE_PA.loc[:, 'leucos'] = pd.to_numeric(SENSE_PA['leucos'], errors='coerce')

promedio_leu = SENSE_PA['leucos'].mean()
desviacion_estandar_leu = SENSE_PA['leucos'].std()

resultado = f"{promedio_leu:.2f} ± {desviacion_estandar_leu:.2f}"
print("Mitjana ± Desviació Estàndard dels leucòcits:", resultado)

Mitjana ± Desviació Estàndard dels leucòcits: 11.52 ± 6.52


#### Pvalor 

##### Normalitat

In [533]:
leuambpa=AMB_PA['leucos'].dropna()
stat, p_value = shapiro(leuambpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8482
Valor p = 0.09127190
El nombre de files segueix una distribució normal.


In [534]:
leuambpamecvv=AMB_PA_MECVV['leucos'].dropna()
stat, p_value = shapiro(leuambpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8811
Valor p = 0.32754766
El nombre de files segueix una distribució normal.


In [535]:
leusensepa=SENSE_PA['leucos'].dropna()
stat, p_value = shapiro(leusensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8178
Valor p = 0.00096236
El nombre de files no segueix una distribució normal.


In [536]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['leucos'], AMB_PA_MECVV['leucos'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [537]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['leucos'], SENSE_PA['leucos'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [538]:
t_stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['leucos'], SENSE_PA['leucos'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Limfòcits %

In [539]:
AMB_PA.loc[:, 'limfos'] = pd.to_numeric(AMB_PA['limfos'], errors='coerce')

promedio_limf = AMB_PA['limfos'].mean()
desviacion_estandar_limf = AMB_PA['limfos'].std()

resultado = f"{promedio_limf:.2f} ± {desviacion_estandar_limf:.2f}"
print("Mitjana ± Desviació Estàndard dels limfòcits %:", resultado)

Mitjana ± Desviació Estàndard dels limfòcits %: 16.70 ± 12.29


In [540]:
AMB_PA_MECVV.loc[:, 'limfos'] = pd.to_numeric(AMB_PA_MECVV['limfos'], errors='coerce')

promedio_limf = AMB_PA_MECVV['limfos'].mean()
desviacion_estandar_limf = AMB_PA_MECVV['limfos'].std()

resultado = f"{promedio_limf:.2f} ± {desviacion_estandar_limf:.2f}"
print("Mitjana ± Desviació Estàndard dels limfòcits %:", resultado)

Mitjana ± Desviació Estàndard dels limfòcits %: 9.57 ± 2.10


In [541]:
SENSE_PA.loc[:, 'limfos'] = pd.to_numeric(SENSE_PA['limfos'], errors='coerce')

promedio_limf = SENSE_PA['limfos'].mean()
desviacion_estandar_limf = SENSE_PA['limfos'].std()

resultado = f"{promedio_limf:.2f} ± {desviacion_estandar_limf:.2f}"
print("Mitjana ± Desviació Estàndard dels limfòcits %:", resultado)

Mitjana ± Desviació Estàndard dels limfòcits %: 19.50 ± 11.57


#### Pvalor

##### Normalitat

In [542]:
limfambpa=AMB_PA['limfos'].dropna()
stat, p_value = shapiro(limfambpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8208
Valor p = 0.04757269
El nombre de files no segueix una distribució normal.


In [543]:
limfambpamecvv=AMB_PA_MECVV['limfos'].dropna()
stat, p_value = shapiro(limfambpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9992
Valor p = 0.94750571
El nombre de files segueix una distribució normal.


In [544]:
limfsensepa=SENSE_PA['limfos'].dropna()
stat, p_value = shapiro(limfsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9290
Valor p = 0.13154504
El nombre de files segueix una distribució normal.


In [545]:
stat_12, p_val_12 = mannwhitneyu(AMB_PA['limfos'], AMB_PA_MECVV['limfos'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [546]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['limfos'], SENSE_PA['limfos'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [547]:
stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['limfos'], SENSE_PA['limfos'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Proteïna C Reactiva

In [548]:
AMB_PA.loc[:, 'prot C react'] = pd.to_numeric(AMB_PA['prot C react'], errors='coerce')

promedio_protcr = AMB_PA['prot C react'].mean()
desviacion_estandar_protcr = AMB_PA['prot C react'].std()

resultado = f"{promedio_protcr:.2f} ± {desviacion_estandar_protcr:.2f}"
print("Mitjana ± Desviació Estàndard de la proteïna C reactiva:", resultado)

Mitjana ± Desviació Estàndard de la proteïna C reactiva: 8.90 ± 5.47


In [549]:
AMB_PA_MECVV.loc[:, 'prot C react'] = pd.to_numeric(AMB_PA_MECVV['prot C react'], errors='coerce')

promedio_protcr = AMB_PA_MECVV['prot C react'].mean()
desviacion_estandar_protcr = AMB_PA_MECVV['prot C react'].std()

resultado = f"{promedio_protcr:.2f} ± {desviacion_estandar_protcr:.2f}"
print("Mitjana ± Desviació Estàndard de la proteïna C reactiva:", resultado)

Mitjana ± Desviació Estàndard de la proteïna C reactiva: 6.38 ± 3.51


In [550]:
SENSE_PA.loc[:, 'prot C react'] = pd.to_numeric(SENSE_PA['prot C react'], errors='coerce')

promedio_protcr = SENSE_PA['prot C react'].mean()
desviacion_estandar_protcr = SENSE_PA['prot C react'].std()

resultado = f"{promedio_protcr:.2f} ± {desviacion_estandar_protcr:.2f}"
print("Mitjana ± Desviació Estàndard de la proteïna C reactiva:", resultado)

Mitjana ± Desviació Estàndard de la proteïna C reactiva: 6.16 ± 6.19


#### Pvalor

##### Normalitat

In [551]:
pcrambpa=AMB_PA['prot C react'].dropna()
stat, p_value = shapiro(pcrambpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9423
Valor p = 0.60665876
El nombre de files segueix una distribució normal.


In [552]:
pcrambpamecvv=AMB_PA_MECVV['prot C react'].dropna()
stat, p_value = shapiro(pcrambpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8067
Valor p = 0.13058912
El nombre de files segueix una distribució normal.


In [553]:
pcrsensepa=SENSE_PA['prot C react'].dropna()
stat, p_value = shapiro(pcrsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8698
Valor p = 0.00636559
El nombre de files no segueix una distribució normal.


In [554]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['prot C react'], AMB_PA_MECVV['prot C react'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [555]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['prot C react'], SENSE_PA['prot C react'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [556]:
t_stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['prot C react'], SENSE_PA['prot C react'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Urea 

In [557]:
AMB_PA.loc[:, 'urea'] = pd.to_numeric(AMB_PA['urea'], errors='coerce')

promedio_urea = AMB_PA['urea'].mean()
desviacion_estandar_urea = AMB_PA['urea'].std()

resultado = f"{promedio_urea:.2f} ± {desviacion_estandar_urea:.2f}"
print("Mitjana ± Desviació Estàndard de la urea:", resultado)

Mitjana ± Desviació Estàndard de la urea: 62.17 ± 36.37


In [558]:
AMB_PA_MECVV.loc[:, 'urea'] = pd.to_numeric(AMB_PA_MECVV['urea'], errors='coerce')

promedio_urea = AMB_PA_MECVV['urea'].mean()
desviacion_estandar_urea = AMB_PA_MECVV['urea'].std()

resultado = f"{promedio_urea:.2f} ± {desviacion_estandar_urea:.2f}"
print("Mitjana ± Desviació Estàndard de la urea:", resultado)

Mitjana ± Desviació Estàndard de la urea: 82.27 ± 32.15


In [559]:
SENSE_PA.loc[:, 'urea'] = pd.to_numeric(SENSE_PA['urea'], errors='coerce')

promedio_urea = SENSE_PA['urea'].mean()
desviacion_estandar_urea = SENSE_PA['urea'].std()

resultado = f"{promedio_urea:.2f} ± {desviacion_estandar_urea:.2f}"
print("Mitjana ± Desviació Estàndard de la urea:", resultado)

Mitjana ± Desviació Estàndard de la urea: 55.03 ± 30.66


#### Pvalor

##### Normalitat

In [560]:
uambpa=AMB_PA['urea'].dropna()
stat, p_value = shapiro(uambpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8842
Valor p = 0.17374314
El nombre de files segueix una distribució normal.


In [561]:
uambpamecvv=AMB_PA_MECVV['urea'].dropna()
stat, p_value = shapiro(uambpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9999
Valor p = 0.97770778
El nombre de files segueix una distribució normal.


In [562]:
usensepa=SENSE_PA['urea'].dropna()
stat, p_value = shapiro(usensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8289
Valor p = 0.00115364
El nombre de files no segueix una distribució normal.


In [563]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['urea'], AMB_PA_MECVV['urea'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [564]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['urea'], SENSE_PA['urea'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [565]:
t_stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['urea'], SENSE_PA['urea'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Filtrat glomerular CDK-EPI

In [566]:
AMB_PA.loc[:, 'FGE CDK-EPI'] = pd.to_numeric(AMB_PA['FGE CDK-EPI'], errors='coerce')

promedio_fg = AMB_PA['FGE CDK-EPI'].mean()
desviacion_estandar_fg = AMB_PA['FGE CDK-EPI'].std()

resultado = f"{promedio_fg:.2f} ± {desviacion_estandar_fg:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI:", resultado)

Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI: 56.00 ± 12.65


In [567]:
AMB_PA_MECVV.loc[:, 'FGE CDK-EPI'] = pd.to_numeric(AMB_PA_MECVV['FGE CDK-EPI'], errors='coerce')

promedio_fg = AMB_PA_MECVV['FGE CDK-EPI'].mean()
desviacion_estandar_fg = AMB_PA_MECVV['FGE CDK-EPI'].std()

resultado = f"{promedio_fg:.2f} ± {desviacion_estandar_fg:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI:", resultado)

Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI: 48.00 ± 2.00


In [568]:
SENSE_PA.loc[:, 'FGE CDK-EPI'] = pd.to_numeric(SENSE_PA['FGE CDK-EPI'], errors='coerce')

promedio_fg = SENSE_PA['FGE CDK-EPI'].mean()
desviacion_estandar_fg = SENSE_PA['FGE CDK-EPI'].std()

resultado = f"{promedio_fg:.2f} ± {desviacion_estandar_fg:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI:", resultado)

Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI: 58.56 ± 18.40


#### Pvalor

##### Normalitat

In [569]:
fgepiambpa=AMB_PA['FGE CDK-EPI'].dropna()
stat, p_value = shapiro(fgepiambpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.7690
Valor p = 0.01994941
El nombre de files no segueix una distribució normal.


In [570]:
fgepiambpamecvv=AMB_PA_MECVV['FGE CDK-EPI'].dropna()
stat, p_value = shapiro(fgepiambpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 1.0000
Valor p = 1.00000000
El nombre de files segueix una distribució normal.


In [571]:
fgepisensepa=SENSE_PA['FGE CDK-EPI'].dropna()
stat, p_value = shapiro(fgepisensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9160
Valor p = 0.14543070
El nombre de files segueix una distribució normal.


In [572]:
t_stat_12, p_val_12 = mannwhitneyu(AMB_PA['FGE CDK-EPI'], AMB_PA_MECVV['FGE CDK-EPI'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [573]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['FGE CDK-EPI'], SENSE_PA['FGE CDK-EPI'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_13)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [574]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['FGE CDK-EPI'], SENSE_PA['FGE CDK-EPI'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Filtrat glomerular MDRD

In [575]:
AMB_PA.loc[:, 'FGE MDRD'] = pd.to_numeric(AMB_PA['FGE MDRD'], errors='coerce')

promedio_fgmd = AMB_PA['FGE MDRD'].mean()
desviacion_estandar_fgmd = AMB_PA['FGE MDRD'].std()

resultado = f"{promedio_fgmd:.2f} ± {desviacion_estandar_fgmd:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular MDRD:", resultado)

Mitjana ± Desviació Estàndard de la filtració glomerular MDRD: 51.50 ± 0.71


In [576]:
AMB_PA_MECVV.loc[:, 'FGE MDRD'] = pd.to_numeric(AMB_PA_MECVV['FGE MDRD'], errors='coerce')

promedio_fgmd = AMB_PA_MECVV['FGE MDRD'].mean()
desviacion_estandar_fgmd = AMB_PA_MECVV['FGE MDRD'].std()

resultado = f"{promedio_fgmd:.2f} ± {desviacion_estandar_fgmd:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular MDRD:", resultado)

Mitjana ± Desviació Estàndard de la filtració glomerular MDRD: 51.00 ± nan


In [577]:
SENSE_PA.loc[:, 'FGE MDRD'] = pd.to_numeric(SENSE_PA['FGE MDRD'], errors='coerce')

promedio_fgmd = SENSE_PA['FGE MDRD'].mean()
desviacion_estandar_fgmd = SENSE_PA['FGE MDRD'].std()

resultado = f"{promedio_fgmd:.2f} ± {desviacion_estandar_fgmd:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular MDRD:", resultado)

Mitjana ± Desviació Estàndard de la filtració glomerular MDRD: 37.67 ± 17.43


#### Pvalor

##### Normalitat

In [578]:
fgmd = AMB_PA['FGE MDRD'].dropna()
stat, p_value = shapiro(fgmd)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")


ValueError: Data must be at least length 3.

In [579]:
stat, p_value = shapiro(AMB_PA_MECVV['FGE MDRD'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

ValueError: Data must be at least length 3.

In [580]:
stat, p_value = shapiro(SENSE_PA['FGE MDRD'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8324
Valor p = 0.11252528
El nombre de files segueix una distribució normal.


In [581]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['FGE MDRD'], AMB_PA_MECVV['FGE MDRD'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [582]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['FGE MDRD'], SENSE_PA['FGE MDRD'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_13)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [583]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['FGE MDRD'], SENSE_PA['FGE MDRD'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i AMB_PA_MECVV:", p_val_23)

P-valor entre AMB_PA_MECVV i AMB_PA_MECVV: nan
